# =============================================================================
# BENCHMARK EVALUATION NOTEBOOK
# =============================================================================
## Purpose:
    - Load the baseline model, processed test data, and the best action determined by the RL agent.
    - Apply the baseline configuration, manual optimization strategies (pruning, quantization), and the agent-discovered optimization strategy to the baseline model.
    - Perform a comprehensive benchmark evaluation of all model versions using standardized metrics (accuracy, parameters, FLOPs, energy consumption).
    - Visualize the comparative performance results.
# =============================================================================

# === Clone Repository & Install Dependencies ===

In [ ]:
!rm -rf Sustainable_AI_Agent_Project
!git clone https://github.com/trongjhuongwr/Sustainable_AI_Agent_Project.git
%cd Sustainable_AI_Agent_Project

In [ ]:
!pip install -q --extra-index-url https://download.pytorch.org/whl/cu121 -r /kaggle/working/Sustainable_AI_Agent_Project/requirements.txt

# 1. Import Libraries and Configuration

In [ ]:
import os
import warnings
import logging
import json
import copy
import random
from tqdm.notebook import tqdm

# Suppress specific warnings for cleaner output
os.environ["GYM_DISABLE_WARNINGS"] = "true"
warnings.filterwarnings("ignore", module="gymnasium")
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger("gymnasium").setLevel(logging.ERROR)
logging.getLogger("stable_baselines3").setLevel(logging.ERROR)

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import gymnasium as gym # Required for potential agent loading fallback
from gymnasium import spaces # Required for potential agent loading fallback
from sklearn.metrics import accuracy_score
from stable_baselines3 import PPO # Required for potential agent loading fallback
from codecarbon import EmissionsTracker
from ptflops import get_model_complexity_info
import torch_pruning as tp
import matplotlib.pyplot as plt
import seaborn as sns
from builtins import print as builtin_print # Avoid tqdm print conflicts

print("Libraries imported successfully.")

# 2. Configuration Class

In [ ]:
# Defines hyperparameters, file paths, and evaluation parameters.
class Config:
    # --- Input Paths ---
    PROCESSED_DATA_PATH = '/kaggle/working/processed_data.pt' # Path to processed data tensors
    BASELINE_MODEL_PATH = '/kaggle/working/baseline_model.pth' # Path to baseline model state dictionary
    AGENT_SAVE_PATH = "/kaggle/working/sustainable_ai_agent_ppo.zip" # Path to saved PPO agent (fallback)
    BEST_ACTION_PATH = "/kaggle/working/best_action.json" # Preferred path for best action info

    # --- Data Parameters ---
    SEQUENCE_LENGTH = 30
    INPUT_DIM = 4

    # --- Model Architecture Parameters (must match baseline) ---
    HIDDEN_DIM = 256
    N_LAYERS = 2
    OUTPUT_DIM = 1
    DROPOUT = 0.2
    SEED = 42 # For reproducibility if agent needs loading

    # --- Evaluation Parameters ---
    EVAL_BATCH_SIZE = 64
    CODECARBON_BATCHES = 10 # Batches for energy measurement

    # --- Computation Device ---
    # Evaluation typically runs on CPU for consistent energy metrics,
    # but DEVICE might be needed if loading the agent requires GPU context.
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds (primarily relevant if agent needs to be loaded and predict)
random.seed(Config.SEED)
np.random.seed(Config.SEED)
torch.manual_seed(Config.SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Config.SEED)
    torch.cuda.manual_seed_all(Config.SEED)

print(f"Configuration loaded. Evaluation device preference: CPU for metrics.")
print(f"Loading data from: {Config.PROCESSED_DATA_PATH}")
print(f"Loading baseline model from: {Config.BASELINE_MODEL_PATH}")
print(f"Attempting to load best action from: {Config.BEST_ACTION_PATH}")
print(f"Agent path (fallback): {Config.AGENT_SAVE_PATH}")

# 3. Utility Functions and Model Definition

In [ ]:
# Replicated from previous notebooks for standalone execution capability.

def count_parameters(model):
    """Counts trainable parameters."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def evaluate_model(model, loader, codecarbon_batches=10):
    """Comprehensive evaluation (accuracy, params, flops, energy). Forces CPU execution."""
    model_cpu = copy.deepcopy(model).cpu()
    device = torch.device("cpu")
    model_cpu.eval()

    # Accuracy
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model_cpu(inputs)
            preds = (outputs > 0.5).float()
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    accuracy = accuracy_score(y_true, y_pred)

    # Energy and CO2 with CodeCarbon
    energy_kwh = 0
    co2_eq_kg = 0
    try:
        tracker = EmissionsTracker(log_level="error", output_dir="/kaggle/working/", tracking_mode="process")
        tracker.start()
        with torch.no_grad():
            # Use tqdm for energy measurement loop visualization
            energy_eval_bar = tqdm(loader, desc="Measuring Energy", total=codecarbon_batches, leave=False)
            batch_count = 0
            for inputs, _ in energy_eval_bar:
                if batch_count >= codecarbon_batches: break
                model_cpu(inputs.to(device))
                batch_count += 1
        tracker.stop()
        if tracker.final_emissions_data:
             energy_kwh = tracker.final_emissions_data.energy_consumed or 0
             co2_eq_kg = tracker.final_emissions_data.emissions or 0
        else:
            builtin_print("Warning: CodeCarbon tracker did not record final emissions data.")
    except Exception as e:
        builtin_print(f"Warning: CodeCarbon measurement failed - {e}")

    # Params
    params = count_parameters(model_cpu)

    # FLOPs with ptflops
    flops = 0
    try:
        macs, _ = get_model_complexity_info(
            model_cpu, (Config.SEQUENCE_LENGTH, Config.INPUT_DIM),
            as_strings=False, print_per_layer_stat=False, verbose=False)
        flops = macs * 2
    except (KeyError, AttributeError, RuntimeError, TypeError):
        # builtin_print("Note: Could not calculate FLOPs via ptflops (likely quantized model). Reporting 0.")
        flops = 0 # Report 0 for quantized models

    return {
        "accuracy": accuracy,
        "energy_kwh": energy_kwh,
        "co2_eq_kg": co2_eq_kg,
        "flops": max(0.0, flops),
        "params": max(0.0, params)
    }

def apply_l1_pruning(model, amount):
    """Applies L1 pruning, ignoring GRU layers. Returns CPU model."""
    if not (0 < amount <= 1.0): return copy.deepcopy(model).cpu()
    model_to_prune = copy.deepcopy(model).cpu()
    model_to_prune.eval()
    ignored_layers = [m for m in model_to_prune.modules() if isinstance(m, nn.GRU)]
    example_inputs = torch.randn(1, Config.SEQUENCE_LENGTH, Config.INPUT_DIM)
    try:
        pruner = tp.pruner.MagnitudePruner(
            model_to_prune, example_inputs,
            importance=tp.importance.MagnitudeImportance(p=1),
            pruning_ratio=amount, ignored_layers=ignored_layers)
        pruner.step()
    except Exception as e:
        builtin_print(f"Error during pruning: {e}")
        return copy.deepcopy(model).cpu()
    return model_to_prune

def apply_dynamic_quantization(model):
    """Applies dynamic quantization. Returns CPU model."""
    try:
        quantized_model = copy.deepcopy(model).cpu()
        quantized_model.eval()
        quantized_model = torch.quantization.quantize_dynamic(
            quantized_model, {nn.GRU, nn.Linear}, dtype=torch.qint8)
        return quantized_model
    except Exception as e:
         builtin_print(f"Error during dynamic quantization: {e}")
         return copy.deepcopy(model).cpu()

class WeatherGRU(nn.Module):
    """GRU model definition."""
    def __init__(self, input_dim, hidden_dim, n_layers, output_dim, dropout):
        super(WeatherGRU, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers,
                          batch_first=True, dropout=dropout if n_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        gru_out, _ = self.gru(x)
        out = self.fc(gru_out[:, -1, :])
        return self.sigmoid(out)

print("Utility functions and WeatherGRU class defined.")

# 4. Load Baseline Model and Test Data

In [ ]:
# Loads the required artifacts: processed test data and the baseline model state.

# Load processed test data tensors
try:
    processed_data = torch.load(Config.PROCESSED_DATA_PATH, map_location=torch.device('cpu')) # Load directly to CPU
    X_test_tensor = processed_data['X_test']
    y_test_tensor = processed_data['y_test']
    builtin_print(f"Processed test data loaded from {Config.PROCESSED_DATA_PATH}")
except FileNotFoundError:
    builtin_print(f"Error: Processed data file not found at {Config.PROCESSED_DATA_PATH}. Please run train_baseline.ipynb first.")
    raise
except Exception as e:
    builtin_print(f"Error loading processed data: {e}")
    raise

# Create Test DataLoader
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# Use a larger batch size for potentially faster evaluation if memory allows
test_loader = DataLoader(test_dataset, batch_size=Config.EVAL_BATCH_SIZE * 2, shuffle=False)
builtin_print("Test DataLoader created.")

# Initialize the GRU model structure
baseline_model_structure = WeatherGRU(
    input_dim=Config.INPUT_DIM,
    hidden_dim=Config.HIDDEN_DIM,
    n_layers=Config.N_LAYERS,
    output_dim=Config.OUTPUT_DIM,
    dropout=Config.DROPOUT
)

# Load the trained baseline model state dictionary onto the CPU
try:
    baseline_model_structure.load_state_dict(torch.load(Config.BASELINE_MODEL_PATH, map_location=torch.device('cpu')))
    baseline_model_structure.eval() # Set to evaluation mode
    builtin_print(f"Baseline model state loaded from {Config.BASELINE_MODEL_PATH}")
except FileNotFoundError:
    builtin_print(f"Error: Baseline model file not found at {Config.BASELINE_MODEL_PATH}. Please run train_baseline.ipynb first.")
    raise
except Exception as e:
    builtin_print(f"Error loading baseline model state: {e}")
    raise

# Keep a clean copy of the loaded baseline model on the CPU
baseline_model_cpu = baseline_model_structure.cpu()
print("Baseline model ready on CPU.")

# 5. Determine Best Action from Agent

In [ ]:
# Loads the best action details (pruning ratio, quantization) determined by the RL agent.
# It prioritizes loading from the JSON file saved by train_rl_agent.ipynb.
# If the JSON is unavailable, it attempts to load the saved PPO agent and predict the action.

best_action_info = None

# --- Attempt 1: Load from JSON file ---
try:
    with open(Config.BEST_ACTION_PATH, 'r') as f:
        best_action_info = json.load(f)
        # Basic validation of the loaded data
        if 'pruning_amount' in best_action_info and 'quantized' in best_action_info:
             builtin_print(f"Best action info successfully loaded from {Config.BEST_ACTION_PATH}:")
             builtin_print(f"  Source: {best_action_info.get('source', 'Unknown')}")
             builtin_print(f"  Pruning: {best_action_info['pruning_amount']*100:.0f}%")
             builtin_print(f"  Quantized: {best_action_info['quantized']}")
             if 'action_code' in best_action_info:
                 builtin_print(f"  Originating Action Code: {best_action_info['action_code']}")
        else:
             builtin_print(f"Warning: JSON file {Config.BEST_ACTION_PATH} is missing required keys ('pruning_amount', 'quantized'). Will attempt agent loading.")
             best_action_info = None # Invalidate if keys are missing
except FileNotFoundError:
    builtin_print(f"Info: Best action JSON file not found at {Config.BEST_ACTION_PATH}. Proceeding to load agent for prediction.")
except json.JSONDecodeError:
     builtin_print(f"Error: Could not decode JSON from {Config.BEST_ACTION_PATH}. File might be corrupted. Proceeding to load agent.")
except Exception as e:
    builtin_print(f"Error loading best action JSON: {e}. Proceeding to load agent.")

# --- Attempt 2: Load Agent and Predict (Fallback) ---
if best_action_info is None:
    builtin_print(f"\n--- Attempting to load trained agent from {Config.AGENT_SAVE_PATH} to predict best action ---")
    try:
        # Define necessary parameters for a minimal environment mock, needed for PPO.load()
        dummy_pruning_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
        dummy_action_space = spaces.Discrete(len(dummy_pruning_levels) * 2)
        dummy_obs_space = spaces.Box(low=np.array([0.0, -1.0, 0.0, 0.0]), high=np.array([1.0, 1.0, 1.0, 1.0]), dtype=np.float32)

        # Minimal mock environment class
        class MockEnv(gym.Env):
            def __init__(self):
                super().__init__(); self.action_space = dummy_action_space; self.observation_space = dummy_obs_space
            def step(self, action): pass
            def reset(self, seed=None, options=None):
                super().reset(seed=seed); return np.array([0.6, 0.0, 0.0, 0.0], dtype=np.float32), {} # Plausible initial state
            def render(self): pass
            def close(self): pass

        mock_env_instance = MockEnv()

        # Load the PPO agent onto the CPU
        agent = PPO.load(Config.AGENT_SAVE_PATH, env=mock_env_instance, device='cpu')
        builtin_print("Agent loaded successfully onto CPU.")

        # Get initial observation from the mock environment
        obs, _ = mock_env_instance.reset()

        # Predict the action deterministically using the loaded agent's policy
        action_pred, _ = agent.predict(obs, deterministic=True)
        action_code_pred = action_pred.item()
        builtin_print(f"Agent predicted best action code: {action_code_pred}")

        # Decode the predicted action code
        n_pruning_levels = len(dummy_pruning_levels)
        pruning_idx_pred = action_code_pred % n_pruning_levels
        pruning_amount_pred = dummy_pruning_levels[pruning_idx_pred]
        quantized_pred = action_code_pred >= n_pruning_levels

        # Store the derived action information
        best_action_info = {
            'source': 'agent_prediction_fallback',
            'action_code': action_code_pred,
            'pruning_amount': pruning_amount_pred,
            'quantized': quantized_pred
        }
        builtin_print(f"  Decoded Action -> Pruning: {pruning_amount_pred*100:.0f}%, Quantized: {quantized_pred}")

    except FileNotFoundError:
        builtin_print(f"Error: Agent file not found at {Config.AGENT_SAVE_PATH}. Cannot evaluate agent-optimized solution.")
        best_action_info = None # Ensure it remains None if agent loading fails
    except Exception as e:
        builtin_print(f"Error loading agent or predicting action: {e}. Cannot evaluate agent-optimized solution.")
        best_action_info = None

# Final check
if best_action_info is None:
     builtin_print("\nWarning: Could not determine the agent's best action from JSON or agent prediction.")

# 6. Run Full Benchmark Comparison

In [ ]:
# This function orchestrates the evaluation of four model versions:
# 1. Baseline: The original trained GRU model.
# 2. Manual Pruning (50%): Baseline model with 50% L1 pruning applied.
# 3. Manual Quantization: Baseline model with dynamic quantization applied.
# 4. Agent Optimized: Baseline model with the pruning/quantization strategy determined in the previous step.
# It uses the `evaluate_model` function for consistent metric calculation on the CPU.

def run_full_benchmark(baseline_model_cpu, test_loader, best_action_info):
    """Runs and aggregates benchmark results for different model versions."""
    results = {} # Dictionary to store metrics for each version
    print("\n--- Starting Full Benchmark Evaluation ---")

    # --- 1. Baseline Evaluation ---
    print("\n--- Evaluating: Baseline Model ---")
    try:
        results['Baseline'] = evaluate_model(copy.deepcopy(baseline_model_cpu), test_loader, Config.CODECARBON_BATCHES)
        print("  Baseline evaluation complete.")
    except Exception as e:
        print(f"  Error evaluating baseline: {e}")
        results['Baseline'] = {} # Store empty dict on error

    # --- 2. Manual Pruning (50%) Evaluation ---
    print("\n--- Evaluating: Manual Pruning (50%) ---")
    try:
        pruned_model_manual = apply_l1_pruning(baseline_model_cpu, 0.5)
        results['Manual Pruning (50%)'] = evaluate_model(pruned_model_manual, test_loader, Config.CODECARBON_BATCHES)
        print("  Manual pruning evaluation complete.")
    except Exception as e:
        print(f"  Error evaluating manual pruning: {e}")
        results['Manual Pruning (50%)'] = {}

    # --- 3. Manual Quantization Evaluation ---
    print("\n--- Evaluating: Manual Quantization ---")
    try:
        quantized_model_manual = apply_dynamic_quantization(baseline_model_cpu)
        results['Manual Quantization'] = evaluate_model(quantized_model_manual, test_loader, Config.CODECARBON_BATCHES)
        print("  Manual quantization evaluation complete.")
    except Exception as e:
        print(f"  Error evaluating manual quantization: {e}")
        results['Manual Quantization'] = {}

    # --- 4. Agent Optimized Evaluation ---
    print("\n--- Evaluating: Agent Optimized Model ---")
    if best_action_info and 'pruning_amount' in best_action_info and 'quantized' in best_action_info:
        pruning_amount_agent = best_action_info['pruning_amount']
        apply_quant_agent = best_action_info['quantized']
        action_code_agent = best_action_info.get('action_code', 'N/A') # Get action code if available
        print(f"  Applying Agent's strategy (Action: {action_code_agent}): Pruning {pruning_amount_agent*100:.0f}%, Quantized: {apply_quant_agent}")
        try:
            agent_optimized_model = copy.deepcopy(baseline_model_cpu) # Start fresh
            if pruning_amount_agent > 0:
                agent_optimized_model = apply_l1_pruning(agent_optimized_model, pruning_amount_agent)
            if apply_quant_agent:
                agent_optimized_model = apply_dynamic_quantization(agent_optimized_model)

            # Evaluate the resulting agent-optimized model
            results['Agent Optimized'] = evaluate_model(agent_optimized_model, test_loader, Config.CODECARBON_BATCHES)
            print("  Agent optimized evaluation complete.")
        except Exception as e:
            print(f"  Error evaluating agent optimized model: {e}")
            results['Agent Optimized'] = {}
    else:
        print("  Skipping agent evaluation: Best action information is not available or invalid.")
        results['Agent Optimized'] = {} # Ensure the key exists even if skipped

    # --- Format Results into a DataFrame ---
    df_results = pd.DataFrame(results).T.fillna(0) # Transpose and fill NaNs

    # Calculate derived metrics (GFLOPs, MParams, mWh) safely
    df_results['flops_g'] = (df_results['flops'] / 1e9).round(6) if 'flops' in df_results else 0.0
    df_results['params_m'] = (df_results['params'] / 1e6).round(6) if 'params' in df_results else 0.0
    df_results['energy_mwh'] = (df_results['energy_kwh'] * 1000).round(6) if 'energy_kwh' in df_results else 0.0

    # Ensure all desired columns exist, adding missing ones with default 0.0
    final_columns = ['accuracy', 'params_m', 'flops_g', 'energy_mwh', 'co2_eq_kg']
    for col in final_columns:
        if col not in df_results.columns:
            df_results[col] = 0.0
        # Ensure numeric type after potential fillna(0)
        df_results[col] = pd.to_numeric(df_results[col], errors='coerce').fillna(0.0)


    print("\n--- Benchmark Evaluation Finished ---")
    # Return only the formatted columns of interest
    return df_results[final_columns]

# --- Execute the Benchmark ---
benchmark_results_df = run_full_benchmark(baseline_model_cpu, test_loader, best_action_info)

# --- Display the Results Table ---
print("\n\n--- FINAL COMPREHENSIVE BENCHMARK RESULTS ---")
# Display using markdown for better rendering in notebooks
print(benchmark_results_df.to_markdown())

# 7. Visualize Results

In [ ]:
# Generates bar charts comparing accuracy, model size, computational cost, and energy consumption
# across the different model versions. Also creates a scatter plot (Pareto front visualization)
# showing the trade-off between accuracy and computational cost (FLOPs).

print("\n--- Generating Performance Comparison Visualizations ---")

sns.set_theme(style="whitegrid") # Use seaborn's whitegrid style
fig, axes = plt.subplots(2, 2, figsize=(15, 11)) # Create a 2x2 grid of plots
fig.suptitle('Comprehensive Model Performance Comparison', fontsize=16, weight='bold')

# Define plot details for each metric
metrics_to_plot = [
    {'metric': 'accuracy', 'ax': axes[0, 0], 'palette': 'Greens_r', 'title': 'Model Accuracy', 'ylabel': 'Accuracy Score'},
    {'metric': 'params_m', 'ax': axes[0, 1], 'palette': 'Blues_r', 'title': 'Model Size (Millions of Parameters)', 'ylabel': 'Parameters (M)'},
    {'metric': 'flops_g', 'ax': axes[1, 0], 'palette': 'Oranges_r', 'title': 'Computational Cost (GFLOPs)', 'ylabel': 'GFLOPs'},
    {'metric': 'energy_mwh', 'ax': axes[1, 1], 'palette': 'Reds_r', 'title': 'Inference Energy Consumption (mWh)', 'ylabel': 'Energy (mWh)'}
]

# Create each bar plot
for plot_info in metrics_to_plot:
    metric = plot_info['metric']
    ax = plot_info['ax']
    if metric in benchmark_results_df.columns:
        try:
            # Generate bar plot using seaborn
            sns.barplot(x=benchmark_results_df.index, y=metric, data=benchmark_results_df, ax=ax, palette=plot_info['palette'])
            ax.set_title(plot_info['title'], fontsize=12, weight='bold')
            ax.set_ylabel(plot_info['ylabel'])
            ax.tick_params(axis='x', rotation=15, labelsize=10) # Rotate x-labels slightly
            ax.tick_params(axis='y', labelsize=10)

            # Add value labels specifically for accuracy bars
            if metric == 'accuracy':
                 for index, value in enumerate(benchmark_results_df[metric]):
                      ax.text(index, value + 0.005, f'{value:.4f}', ha='center', va='bottom', fontsize=9) # Position label slightly above bar

            # Adjust y-axis limits for better visualization, handling zero values
            min_val, max_val = benchmark_results_df[metric].min(), benchmark_results_df[metric].max()
            if max_val > min_val: # If there's a range
                 padding = (max_val - min_val) * 0.1 # Add 10% padding
                 ax.set_ylim(bottom=max(0, min_val - padding), top=max_val + padding)
            elif max_val > 0: # If all values are the same and positive
                 ax.set_ylim(bottom=0, top=max_val * 1.2)
            else: # If all values are zero or negative
                 ax.set_ylim(bottom=min_val * 1.1 if min_val < 0 else -0.01, top=0.01)

        except Exception as e:
            builtin_print(f"Warning: Could not plot {metric}: {e}")
            ax.set_title(f"{plot_info['title']}\\n(Plotting Error)", fontsize=12, weight='bold')
    else:
        ax.set_title(f"{plot_info['title']}\\n(Data N/A)", fontsize=12, weight='bold')


plt.tight_layout(rect=[0, 0.03, 1, 0.96]) # Adjust overall layout
plt.show()

# --- Generate Pareto Plot (Accuracy vs. FLOPs) ---
print("\n--- Generating Pareto Plot (Accuracy vs. Computational Cost) ---")
plt.figure(figsize=(10, 6)) # Adjust figure size

if 'flops_g' in benchmark_results_df.columns and 'accuracy' in benchmark_results_df.columns:
    try:
        sns.scatterplot(
            data=benchmark_results_df,
            x='flops_g',
            y='accuracy',
            hue=benchmark_results_df.index,
            s=120, # Marker size
            style=benchmark_results_df.index,
            palette='viridis'
        )
        plt.title('Accuracy vs. Computational Cost (GFLOPs) Trade-off', fontsize=14, weight='bold')
        plt.xlabel('Computational Cost (GFLOPs)', fontsize=11)
        plt.ylabel('Accuracy', fontsize=11)
        plt.legend(title='Model Version', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
        plt.grid(True, linestyle='--', alpha=0.6)

        # Adjust plot limits for clarity, especially for near-zero FLOPs
        max_flops = benchmark_results_df['flops_g'].max()
        plt.xlim(left=-0.002, right=max_flops * 1.1 if max_flops > 0 else 0.002) # Start slightly left of 0

        min_acc, max_acc = benchmark_results_df['accuracy'].min(), benchmark_results_df['accuracy'].max()
        acc_padding = (max_acc - min_acc) * 0.1 if (max_acc - min_acc) > 0.01 else 0.01
        plt.ylim(bottom=min_acc - acc_padding, top=max_acc + acc_padding)

        plt.tight_layout()
        plt.show()
    except Exception as e:
        builtin_print(f"Warning: Could not generate Pareto plot: {e}")
else:
    builtin_print("Skipping Pareto plot: Required columns ('flops_g', 'accuracy') not found in results.")